In [1]:
include("src//graph.jl")

using ..HllGraph
using ..HllSets

using Redis
using EasyConfig
using UUIDs
using Random

conn = Redis.RedisConnection()

RedisConnection("127.0.0.1", 6379, "", 0, Redis.Transport.TCPTransport(Sockets.TCPSocket(RawFD(59) open, 0 bytes waiting)))

In [2]:
# Initialize test HllSets
hll1 = HllSets.HllSet{10}()
hll2 = HllSets.HllSet{10}()
hll3 = HllSets.HllSet{10}()
hll4 = HllSets.HllSet{10}()
hll5 = HllSets.HllSet{10}()

# Generate datasets from random strings
s1 = Set(randstring(7) for _ in 1:10)
s2 = Set(randstring(7) for _ in 1:15)
s3 = Set(randstring(7) for _ in 1:100)
s4 = Set(randstring(7) for _ in 1:20)
s5 = Set(randstring(7) for _ in 1:130)

# Add datasets to HllSets
HllSets.add!(hll1, s1)
HllSets.add!(hll2, s2)
HllSets.add!(hll3, s3)
HllSets.add!(hll4, s4)
HllSets.add!(hll5, s5)

print(hll1)

HllSet{10}()


In [3]:
# Pr9nt cardinality of datasets and HllSets side by side
print(length(s1), " : ", HllSets.count(hll1), "\n")
print(length(s2), " : ", HllSets.count(hll2), "\n")
print(length(s3), " : ", HllSets.count(hll3), "\n")
print(length(s4), " : ", HllSets.count(hll4), "\n")
print(length(s5), " : ", HllSets.count(hll5), "\n\n")

# union
print(length(s1 ∪ s2 ∪ s3 ∪ s4 ∪ s5), " : ", HllSets.count(hll1 ∪ hll2 ∪ hll3 ∪ hll4 ∪ hll5), "\n")

# intersection
print(length(s1 ∩ s2 ∩ s3 ∩ s4 ∩ s5), " : ", HllSets.count(hll1 ∩ hll2 ∩ hll3 ∩ hll4 ∩ hll5), "\n")

10 : 10
15 : 16
100 : 100
20 : 18
130 : 134

275 : 281
0 : 1


In [4]:
node1 = HllGraph.Node(HllSets.id(hll1), ["node1"], HllSets.dump(hll1))
node2 = HllGraph.Node(HllSets.id(hll2), ["node2"], HllSets.dump(hll2))
node3 = HllGraph.Node(HllSets.id(hll3), ["node3"], HllSets.dump(hll3))
node4 = HllGraph.Node(HllSets.id(hll4), ["node4"], HllSets.dump(hll4))
node5 = HllGraph.Node(HllSets.id(hll5), ["node5"], HllSets.dump(hll5))

# println(node1.searchable)

graph = HllGraph.Graph([node1, node2, node3, node4, node5], [])

to_blob: 1024; 4096
to_blob: 1024; 4096
to_blob: 1024; 4096
to_blob: 1024; 4096
to_blob: 1024; 4096


Main.HllGraph.Graph(Main.HllGraph.Node[Node(5c94a3ac94480f7b4c0e40294231b7ada6323388; ["node1"]; 10), Node(abb71c6a26339830f1d58fe63c09e5caae3b3705; ["node2"]; 16), Node(ce7cd9b5aa1aa0f26d571fa21346badb8080bb08; ["node3"]; 100), Node(6648e755fa0d54e4122227b2ab4b81d1b768d6fb; ["node4"]; 18), Node(2cd21d2debc73eeb9b8bffb7532cb08d176b7c95; ["node5"]; 134)], Main.HllGraph.Edge[])

In [5]:
HllGraph.set_node(conn, node1, "b")
HllGraph.set_node(conn, node2, "b")
HllGraph.set_node(conn, node3, "b")
HllGraph.set_node(conn, node4, "b")
HllGraph.set_node(conn, node5, "b")

true

In [6]:
println(HllGraph.get_node(conn, node1.sha1, "b"))
println(node1.sha1)
println(node1.labels)

println(HllGraph.get_nodes(conn, "b"))

println(HllGraph.get_edges(conn, "b"))

to_blob: 1024; 4096
Node(5c94a3ac94480f7b4c0e40294231b7ada6323388; ["node1"]; 10)
5c94a3ac94480f7b4c0e40294231b7ada6323388
["node1"]
to_blob: 1024; 4096
to_blob: 1024; 4096
to_blob: 1024; 4096
to_blob: 1024; 4096
to_blob: 1024; 4096
Any[Node(5c94a3ac94480f7b4c0e40294231b7ada6323388; ["node1"]; 10), Node(abb71c6a26339830f1d58fe63c09e5caae3b3705; ["node2"]; 16), Node(ce7cd9b5aa1aa0f26d571fa21346badb8080bb08; ["node3"]; 100), Node(6648e755fa0d54e4122227b2ab4b81d1b768d6fb; ["node4"]; 18), Node(2cd21d2debc73eeb9b8bffb7532cb08d176b7c95; ["node5"]; 134)]
Any[]


In [7]:
union_node = HllGraph.union_nodes(conn, node1, node2, ["union"], "b")

to_blob: 1024; 4096


Node(32af1fa7da9de76cda55e90d4ddecd9a87462f08; ["union"]; 26)

In [8]:
println(HllGraph.get_edges(conn, "b"))

Any[Edge(5c94a3ac94480f7b4c0e40294231b7ada6323388, 32af1fa7da9de76cda55e90d4ddecd9a87462f08, "union"), Edge(abb71c6a26339830f1d58fe63c09e5caae3b3705, 32af1fa7da9de76cda55e90d4ddecd9a87462f08, "union")]
